In [1]:
## This will copy any and all image folders to our girder instance
import os, sys
from os.path import join as oj
import girder_client


##import DSACBIRHelperFunctions as DSACBIR

class LinePrinter():
    """
    Print things to stdout on one line dynamically
    """
    def __init__(self,data):
        sys.stdout.write("\r\x1b[K"+data.__str__())
        sys.stdout.flush()

In [2]:
gc = girder_client.GirderClient(apiUrl='http://candygram.neurology.emory.edu:8080/api/v1')
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [3]:
imageFolderOnLocalMachine = "/home/dagutman/dev/KerasSimpsons_Tensorflow/rawImageData/"
imageFolderUID = '5b0ebfab92ca9a001733549d'

### The Raw Image / Labelled Data Lives in --> CBIR_Prototypes-->KaggleSimpsonsDataSet-->ImageFolders-->AllImageFolders
### Simpsons References https://www.kaggle.com/paultimothymooney/simpsons-recognition-method-comparison

In [4]:
## This compares files on my local system that I want to SYNC with Girder, and makes sure
## I have uploaded everything... first I am making sure there is a FOLDER on Girder for every
## character

imageFoldersOnDSAServer = list([x['name'] for x in gc.listFolder(imageFolderUID)])
    
foldersOnLocalMachine = os.listdir(imageFolderOnLocalMachine)
## If I have a folder on my local machine but it's not on the DSA server I need to create it
for f in foldersOnLocalMachine:
    if f not in imageFoldersOnDSAServer:
        print "Creating folder %s on the DSA Server " % f
        gc.createFolder(imageFolderUID,f,reuseExisting=True)

In [5]:
### Go through the image folders on the DSA Server and see how many images are in each folder and if all the
### local images are available/uploaded
## This determines which images are already uploaded to Girder, and in the next block I check if it's uploaded
GirderImageInfoDict = {}


imagesProcessed = largeImagesMade = 0

for cf in gc.listFolder(imageFolderUID):  ## cf = characterFolder
    characterImages = list( gc.listItem(cf['_id']))
    print "There are a total of %d items for %s" % (len(characterImages), cf['name'])
    GirderImageInfoDict[cf['name']] = { 'girderFolderID': cf['_id'],'numItems': len(characterImages), 'imageItemData': characterImages,
                                      'imageItemNames' : [x['name'] for x in characterImages]
                                      }
  
    ## Check each image item for appropriate tags.. i.e. characterClass and largeItem
    if len(characterImages) > 0:
        for itm in characterImages:
            imagesProcessed +=1
            if 'largeImage' not in itm.keys():
                #print itm['name'],"Needs to be largeImaged..."
                # Create a large image for this item.
                gc.post('/item/%s/tiles' % itm['_id'])
                largeImagesMade+=1

                
            if 'meta' not in itm.keys():
                ### This definitely needs metadata... need to think of cleanest implementation
                gc.addMetadataToItem(itm['_id'],{'characterClass': cf['name']})
            else:
                if 'characterClass' not in itm['meta'].keys():
                    gc.addMetadataToItem(itm['_id'],{'characterClass': cf['name']})
                    
                
                ### This will BREAK if we add other pieces of metadata
                ### gc.addMetaData
            LinePrinter("A total of %d images have been processed, and %d have been largeImage'd" % (imagesProcessed, largeImagesMade))
    print ### Adds a linefeed between each character folder

    ### I am also going to iterate through each item in Girder, and make sure it has a largeImageTag
    ## And also has characterClass associated with each Item..
    #meta.charClass = charactername

There are a total of 913 items for abraham_grampa_simpson
A total of 913 images have been processed, and 0 have been largeImage'd
There are a total of 42 items for agnes_skinner
A total of 955 images have been processed, and 0 have been largeImage'd
There are a total of 623 items for apu_nahasapeemapetilon
A total of 1578 images have been processed, and 0 have been largeImage'd
There are a total of 106 items for barney_gumble
A total of 1684 images have been processed, and 0 have been largeImage'd
There are a total of 1342 items for bart_simpson
A total of 3026 images have been processed, and 0 have been largeImage'd
There are a total of 0 items for bumblebee_man

There are a total of 98 items for carl_carlson
A total of 3124 images have been processed, and 0 have been largeImage'd
There are a total of 1193 items for charles_montgomery_burns
A total of 4317 images have been processed, and 0 have been largeImage'd
There are a total of 986 items for chief_wiggum
A total of 5303 images ha

In [15]:
### This should only be done once.... I am going to assign training/testing and/or validation labels
### To each Image in each folder, I am also going to add another metadata field called
## setID which refers to where I downloaded the image list, 

### The training set images were placed in here /home/dagutman/dev/KerasSimpsons/simpsons_dataset
GirderImageInfoDict = {}
imagesProcessed = imagesTagged = 0
localTrainingSetDirectory = "/home/dagutman/dev/KerasSimpsons/simpsons_dataset/"

for cf in gc.listFolder(imageFolderUID):  ## cf = characterFolder
    
    ### See if the current character is included in the training set we are using
    localCharTrainingDir = localTrainingSetDirectory+cf['name']
    if os.path.isdir(localCharTrainingDir):
        print len(localCharTrainingDir),"are in the training set for ",cf['name']
        localCharTrainingImages = os.listdir(localCharTrainingDir)
    
    characterImages = list( gc.listItem(cf['_id']))
    print "There are a total of %d items for %s" % (len(characterImages), cf['name'])
    GirderImageInfoDict[cf['name']] = { 'girderFolderID': cf['_id'],'numItems': len(characterImages), 'imageItemData': characterImages,
                                      'imageItemNames' : [x['name'] for x in characterImages]
                                      }
  
    ## Check each image item for appropriate tags.. i.e. training/testing and setID
    for itm in characterImages:
        imagesProcessed +=1
        if 'meta'  in itm.keys():
            if itm['name'] in localCharTrainingImages:
                #print itm['name'], "Should be part of the training data set.."
                ### So I am going to add some metadata now for this image
                imgTags = { 'training': True, 'testing': False, 'validation': False,
                            'trainingSetName': 'kaggle20Class', 'imgSourceId': 'KaggleSimpsons'}
                gc.addMetadataToItem(itm['_id'], imgTags)
            LinePrinter("A total of %d images have been processed, and %d have been tagged with training Set Info" % (imagesProcessed, imagesTagged))
    print ### Adds a linefeed between each character folder


72 are in the training set for  abraham_grampa_simpson
There are a total of 913 items for abraham_grampa_simpson
A total of 913 images have been processed, and 0 have been tagged with training Set Info
There are a total of 42 items for agnes_skinner
A total of 955 images have been processed, and 0 have been tagged with training Set Info
72 are in the training set for  apu_nahasapeemapetilon
There are a total of 623 items for apu_nahasapeemapetilon
A total of 1578 images have been processed, and 0 have been tagged with training Set Info
There are a total of 106 items for barney_gumble
A total of 1684 images have been processed, and 0 have been tagged with training Set Info
62 are in the training set for  bart_simpson
There are a total of 1342 items for bart_simpson
A total of 3026 images have been processed, and 0 have been tagged with training Set Info
There are a total of 0 items for bumblebee_man

There are a total of 98 items for carl_carlson
A total of 3124 images have been process

In [ ]:
###imageFoldersOnDSAServer = list([x['name'] for x in gc.listFolder(imageFolderUID)])
    
foldersOnLocalMachine = os.listdir(imageFolderOnLocalMachine)
## If I have a folder on my local machine but it's not on the DSA server I need to create it
#print len(imageFolderOnLocalMachine),len(foldersOnLocalMachine)

imagesUploaded = imagesSynced = 0

for f in foldersOnLocalMachine:
    print f
    ### Let's look at the images in each directory, and then see if they have been uploaded to girder Yet
    imagesInLocalCharacterFolder = os.listdir(  os.path.join(imageFolderOnLocalMachine, f))
    
    ### now walk through every image on localhost and upload it if it's not in girder..
    for ci in imagesInLocalCharacterFolder:
        imagesSynced +=1
        if ci not in GirderImageInfoDict[f]['imageItemNames']:
            
            gc.uploadFileToFolder( GirderImageInfoDict[f]['girderFolderID'], oj(imageFolderOnLocalMachine,f,ci) )
            imagesUploaded+=1
            
        LinePrinter("A total of %d files have been scanned and %d uploaded " % ( imagesSynced,imagesUploaded))
    ## Check and see if girder already has them, if not upload them...


In [ ]:
### Above are images in Girder, I need to compare to my local filesystem

### Download Directory
## getSimpsonsTrainingData.py  --version=1  --outputDir=SomeDir  --girderServer=<something> 


#python runFelipescode.py --init=>whatever<  
## #overwriteExisting = 


## MY IDEAL WORKFLOW

## Look in README.md

## git clone <repo>
## cd Repo
## python scripts/syncTrainingData.py  <<GETS ALL THE DAT

## Ge the latests docker image

## docker pull Tensorflowthatworksfromsomewhere

## docker run --dataDir:/data:/home/dagutman/datad.kas.kkdf   tensorflowAwesome  --notebookDir=notebooks/ 


## outputs are saved to results/


## we have another interactive script that will upload the results file if you give it some info

## python publishResults.py   ## Defaults to interactive something or other

## results get timestamped/saved automagically









